# Deep Learning dan Shallow Learning
### Muhammad Shafwan Faturrahman (13316006)

In [1]:
#Import data latih angka hingga siap olah 
import numpy as np
from keras.datasets import mnist
(x_num_trn, y_num_trn),(x_num_test, y_num_test)= mnist.load_data()
x_num_all = np.append(x_num_trn,x_num_test,axis=0)
y_num_all = np.append(y_num_trn,y_num_test,axis=0)

Using TensorFlow backend.


In [2]:
#Import data latih huruf(A-J) hingga siap olah
from emnist import extract_training_samples, extract_test_samples
x_alp_trn, y_alp_trn = extract_training_samples('letters')
x_alp_test, y_alp_test = extract_test_samples('letters')
x_alp_trn = np.array(x_alp_trn[np.argwhere(y_alp_trn<=10),:,:])
x_alp_trn = np.squeeze(x_alp_trn, axis = 1)
y_alp_trn = np.array(y_alp_trn[np.argwhere(y_alp_trn<=10)]) + 9
y_alp_trn = np.squeeze(y_alp_trn, axis = 1)
x_alp_test = np.array(x_alp_test[np.argwhere(y_alp_test<=10),:,:])
x_alp_test = np.squeeze(x_alp_test, axis = 1)
y_alp_test = np.array(y_alp_test[np.argwhere(y_alp_test<=10)]) + 9
y_alp_test = np.squeeze(y_alp_test, axis = 1)
x_alp_all = np.append(x_alp_trn,x_alp_test,axis=0)
y_alp_all = np.append(y_alp_trn,y_alp_test,axis=0)

In [3]:
#Data gabungan angka dan huruf
x_all = np.append(x_num_all,x_alp_all,axis=0)
y_all = np.append(y_num_all,y_alp_all,axis=0)

In [4]:
#Data train/test split
from sklearn.model_selection import train_test_split
x_trn, x_test, y_trn, y_test = train_test_split(x_all, y_all, test_size=0.3)

In [5]:
#Data format configuration
import keras.backend as K
img_rows = 28
img_cols = 28
if K.image_data_format() == 'channels_first':
    x_trn = x_trn.reshape(x_trn.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_trn = x_trn.reshape(x_trn.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
x_trn = x_trn.astype('float32')
x_test = x_test.astype('float32')
x_trn /= 255
x_test /= 255
print('x_train shape:', x_trn.shape)
print(x_trn.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (88200, 28, 28, 1)
88200 train samples
37800 test samples


In [6]:
#Classing
import keras.utils as Ku
num_class = 20
y_trn = Ku.to_categorical(y_trn, num_class)
y_test = Ku.to_categorical(y_test, num_class)

In [25]:
#Model Architecture
#Deep Learning (CNN)
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
model_1 = Sequential()
model_1.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model_1.add(Conv2D(64, (3, 3), activation='relu'))
model_1.add(MaxPooling2D(pool_size=(2, 2)))
model_1.add(Dropout(0.25))
model_1.add(Flatten())
model_1.add(Dense(784, activation='relu'))
model_1.add(Dense(392, activation='relu'))
model_1.add(Dense(196, activation='relu'))
model_1.add(Dense(98, activation='relu'))
model_1.add(Dense(49, activation='relu'))
model_1.add(Dropout(0.5))
model_1.add(Dense(num_class, activation='softmax'))
model_1.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])

In [20]:
#Shallow Learning (NN)
model_2 = Sequential()
model_2.add(Flatten(input_shape=input_shape))
model_2.add(Dense(98, activation='relu'))
model_2.add(Dropout(0.5))
model_2.add(Dense(num_class, activation='softmax'))
model_2.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])

In [28]:
#Summary model_1
model_1.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_14 (Dropout)         (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 9216)              0         
_________________________________________________________________
dense_32 (Dense)             (None, 784)               7226128   
_________________________________________________________________
dense_33 (Dense)             (None, 392)             

In [27]:
#Summary model_2
model_2.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_8 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_24 (Dense)             (None, 98)                76930     
_________________________________________________________________
dropout_11 (Dropout)         (None, 98)                0         
_________________________________________________________________
dense_25 (Dense)             (None, 20)                1980      
Total params: 78,910
Trainable params: 78,910
Non-trainable params: 0
_________________________________________________________________


In [29]:
#Training
from tensorflow.keras.callbacks import CSVLogger,EarlyStopping
logfile_cnn = 'MNIST-CNN.log'
logfile_nn = 'MNIST-NN.log'
csv_logger_1 = CSVLogger(logfile_cnn, append=True)  
csv_logger_2 = CSVLogger(logfile_nn, append=True)
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='auto')

In [32]:
#Fit model_1
model_1.fit(x_trn, y_trn,
          batch_size=512,
          epochs=10,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[csv_logger_1,early_stopping]
          )
score = model_1.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 88200 samples, validate on 37800 samples
Epoch 1/10
88200/88200 [==============================] - 349s 4ms/step - loss: 0.5808 - accuracy: 0.8389 - val_loss: 0.2609 - val_accuracy: 0.9215
Epoch 2/10
88200/88200 [==============================] - 344s 4ms/step - loss: 0.2705 - accuracy: 0.9279 - val_loss: 0.1620 - val_accuracy: 0.9508
Epoch 3/10
88200/88200 [==============================] - 356s 4ms/step - loss: 0.1821 - accuracy: 0.9533 - val_loss: 0.1633 - val_accuracy: 0.9553
Epoch 4/10
88200/88200 [==============================] - 346s 4ms/step - loss: 0.1381 - accuracy: 0.9641 - val_loss: 0.1200 - val_accuracy: 0.9679
Epoch 5/10
88200/88200 [==============================] - 348s 4ms/step - loss: 0.1084 - accuracy: 0.9722 - val_loss: 0.1023 - val_accuracy: 0.9733
Epoch 6/10
88200/88200 [==============================] - 345s 4ms/step - loss: 0.0836 - accuracy: 0.9781 - val_loss: 0.0969 - val_accuracy: 0.9784
Epoch 7/10
88200/88200 [==============================] - 344s

In [33]:
#Fit model_2
model_2.fit(x_trn, y_trn,
          batch_size=512,
          epochs=100,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[csv_logger_2,early_stopping]
          )
score = model_2.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 88200 samples, validate on 37800 samples
Epoch 1/100
88200/88200 [==============================] - 3s 31us/step - loss: 0.3822 - accuracy: 0.8834 - val_loss: 0.2267 - val_accuracy: 0.9341
Epoch 2/100
88200/88200 [==============================] - 3s 38us/step - loss: 0.3771 - accuracy: 0.8839 - val_loss: 0.2266 - val_accuracy: 0.9339
Epoch 3/100
88200/88200 [==============================] - 3s 32us/step - loss: 0.3753 - accuracy: 0.8855 - val_loss: 0.2219 - val_accuracy: 0.9343
Epoch 4/100
88200/88200 [==============================] - 3s 35us/step - loss: 0.3707 - accuracy: 0.8869 - val_loss: 0.2229 - val_accuracy: 0.9349
Epoch 5/100
88200/88200 [==============================] - 3s 33us/step - loss: 0.3689 - accuracy: 0.8858 - val_loss: 0.2213 - val_accuracy: 0.9341
Epoch 6/100
88200/88200 [==============================] - 3s 34us/step - loss: 0.3647 - accuracy: 0.8875 - val_loss: 0.2195 - val_accuracy: 0.9353
Epoch 7/100
88200/88200 [==============================] - 3s 

In [34]:
#Pengolahan data input
from PIL import ImageOps as IO
from numpy import asarray
def fix_data(data_input):
  width, height = data_input.size
  left = 0.1*width
  top = 0.1*height
  right = 0.9*width
  bottom = 0.9*height
  cropped_data = data_input.crop((left, top, right, bottom))
  GS_data = IO.grayscale(cropped_data)
  inv_data = IO.invert(GS_data)
  rsz_data = inv_data.resize((28,28))
  fixed_data = asarray(rsz_data)
  return fixed_data

In [35]:
#Import data input
from os import listdir
from PIL import Image
datasets_test = "image-crop"
list_data = listdir(datasets_test)
data_gmb = []
for nama in list_data:
  data_img = Image.open(datasets_test + "/" + nama)
  data_img = fix_data(data_img)
  data_gmb.append(data_img)
data_gmb = asarray(data_gmb)
if K.image_data_format() == 'channels_first':
    data_gmb = data_gmb.reshape(data_gmb.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    data_gmb = data_gmb.reshape(data_gmb.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [36]:
#Prediksi output gambar model_1
pred_gmb_cnn = model_1.predict(data_gmb)
class_gmb_cnn = []
for i in range(len(data_gmb)):
    class_pred_cnn = 0
    for j in range(num_class-1):
        if pred_gmb_cnn[i,j+1] > class_pred_cnn:
            class_pred_cnn = j+1
    class_gmb_cnn.append(class_pred_cnn)

In [37]:
#Prediksi output gambar model_2
pred_gmb_nn = model_2.predict(data_gmb)
class_gmb_nn = []
for i in range(len(data_gmb)): 
    class_pred_nn = 0
    for j in range(num_class-1):
        if pred_gmb_nn[i,j+1] > class_pred_nn:
            class_pred_nn = j+1
    class_gmb_nn.append(class_pred_nn)

In [39]:
#Membuat file prediksi gambar
import pandas as pd
import xlsxwriter as xw
format_data = pd.read_csv('format-prediksi-submit.csv')
wb = xw.Workbook('prediksi-submit.xlsx')
ws = wb.add_worksheet('Jawaban Gambar')
huruf = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
head_table = ['image_filename', 'output model_1', 'output model_2']
for i in range(len(head_table)):
    ws.write(0,i,head_table[i])
pred_model_1 = class_gmb_cnn
pred_model_2 = class_gmb_nn
for i in range(len(data_gmb)):
    if pred_model_1[i]<10:
        jwb_model_1 = int(pred_model_1[i])
    else:
        jwb_model_1 = pred_model_1[i] - 10
        jwb_model_1 = huruf[int(jwb_model_1)]
    if pred_model_2[i]<10:
        jwb_model_2 = int(pred_model_2[i])
    else:
        jwb_model_2 = pred_model_2[i] - 10
        jwb_model_2 = huruf[int(jwb_model_2)]
    ws.write((i+1),0,str(format_data.iloc[i,0]))
    ws.write((i+1),1,jwb_model_1)
    ws.write((i+1),2,jwb_model_2)
wb.close()

In [42]:
#File prediksi MNIST
pred_file = pd.read_excel('prediksi-submit.xlsx')
pred_file

,image_filename,output model_1,output model_2
0,0001-00-00.jpg,0,B
1,0001-00-01.jpg,B,B
2,0001-00-02.jpg,D,D
3,0001-00-03.jpg,D,B
4,0001-00-04.jpg,6,B
...,...,...,...
5515,A101b-09-07.jpg,J,D
5516,A101b-09-08.jpg,J,D
5517,A101b-09-09.jpg,A,D
5518,A101b-09-10.jpg,J,G


# Analisis

   Dalam menentukan perbedaan beberapa karakteristik dari kedua model training maka digunakan Convolutional Neural Network (CNN) model untuk deep learning dan 1 Layer Neural Network (NN) untuk shallow learning. Bentuk architecture kedua model dapat dilihat diatas pada bagian model.summary(). Berdasarkan data yang ditunjukan pada saat training dilakukan dapat dilihat bahwa deep learning dengan metode CNN memiliki kecepatan yang lebih lama daripada shallow learning dengan metode NN. Waktu yang dibutuhkan untuk setiap epochnya dapat dilihat diatas bahwa untuk metode CNN (deep learning) memerlukan waktu kurang lebih sekitar 350 detik, sedangkan untuk metode NN (shallow learning) hanya memerlukan waktu 3 hingga 4 detik saja. Hal ini tentunya dipengaruhi oleh banyaknya parameter yang digunakan pada model architecture yang digunakan. Jumlah parameter yang terbentuk tentunya jauh berbeda sehingga memberikan perbedaan waktu yang cukup signifikan. Hal ini juga yang menjadikan ukuran model CNN (deep learning) tentunya memerlukan memori penyimpanan yang lebih besar daripada NN (shallow learning). Jumlah parameter yang terdapat pada kedua model dapat dilihat sebagai berikut.
   CNN = 7,654,849 params
   NN  = 78,910 params
   Akurasi yang dihasilkan oleh kedua metode tersebut tentunya akan berbeda pula. Validation Accuracy pada metode CNN (deep learning) memiliki angka yang lebih tinggi daripada metode NN (shallow learning). Metode CNN (deep learning) memiliki val_accuracy sebesar 0.9792 sedangkan metode NN (shallow learning) hanya memiliki val_accuracy sebesar  0.9424. Jika dilihat angkanya maka kedua metode tersebut tidak memiliki val_accuracy yang tidak jauh beda, namun hasil yang didapatkan tentunya sangatlah berbeda seperti yang bisa dilihat diatas pada file prediksi yang telah dibuat. Untuk kecepatan prediksi model tidak dapat diketahui berapa lama waktu yang dibutuhkan untuk kedua model, namun pada saat menjalankan perintah prediksi tersebut tidak memerlukan waktu yang lama untuk kedua model. 
   Berdasarkan beberapa data tersebut maka dapat disimpulkan kedua model training memiliki kelebihan masing-masing baik dari kecepatan operasi dan juga akurasi hasil yan didapatkan. Hal itu dapat dilihat dari beberapa parameter yang telah dibahas seperti ukuran model, kecepatan training dan akurasinya.